In [1]:
import pinocchio as pin
import numpy as np
import time
import scipy
from example_robot_data import load

## visualise the robot
from pinocchio.visualize import MeshcatVisualizer

## visualise the polytope and the ellipsoid
import meshcat.geometry as g 

# import pycapacity 
import pycapacity as pycap

import meshcat

In [18]:
def getModelIds(q_shoulder, name):
    return [i for i, q in enumerate(q_shoulder) if name == q[-1].split("_")[0]]
def getGraspIds(grasps, name):
    return [i for i, q in enumerate(grasps) if name == q[-1]]

def reduce_model(robot, tolock, lock_vals=None):
    # Get the ID of all existing joints
    lvals = np.zeros(robot.nq)
    jointsToLockIDs = []
    for i,jn in enumerate(tolock):
        if robot.model.existJointName(jn):
            jointsToLockIDs.append(robot.model.getJointId(jn))
            if lock_vals is not None:
                lvals[i] = lock_vals[i]
    model, collision_model = pin.buildReducedModel(robot.model, robot.collision_model, jointsToLockIDs, lvals)
    data = model.createData()
    return model, data, collision_model


def config_collision_model(robot):
    model = robot.model
    data = robot.model.createData()
    geom_model = robot.collision_model
    for i in [0, 1, 2, 11, 20, 21]: # torso, head, 3x bars
        # geom_model.addCollisionPair(pin.CollisionPair(i,3)) # left upper arm
        # geom_model.addCollisionPair(pin.CollisionPair(i,4)) # left lower arm
        # geom_model.addCollisionPair(pin.CollisionPair(i,5)) # left wrist
        # geom_model.addCollisionPair(pin.CollisionPair(i,6)) # left palm
        geom_model.addCollisionPair(pin.CollisionPair(i,12)) # right upper arm
        geom_model.addCollisionPair(pin.CollisionPair(i,13)) #  right lower arm
        geom_model.addCollisionPair(pin.CollisionPair(i,14)) #right  wrist
        geom_model.addCollisionPair(pin.CollisionPair(i,15)) # left palm
    geom_data = pin.GeometryData(geom_model)
    robot.rebuildData()
    return geom_model, geom_data

def check_rechable_with_collisions(robot, geom_model, theta, getjoints, N=15):

    if len(theta)==0:
        return False, np.zeros(robot.nq)

    for t in np.linspace(theta[0],theta[1],N):
        q, _ = getjoints(t)
        if np.all(q <= robot.model.upperPositionLimit) and np.all(q >= robot.model.lowerPositionLimit):
            is_collision = computeCollisions(robot, geom_model, q, False)
            if not is_collision:
                return True, q
    return False, np.zeros(robot.nq)


def getJointNames(robot):
    names = []
    for j in robot.model.frames[1:]:
        if robot.index(j.name) <= robot.nq:
            names.append(j.name)
    return names

def computeCollisions(robot, geom_model, q, verbose = True):
    geom_data = geom_model.createData()
    has_collision = pin.computeCollisions(robot.model, robot.data, geom_model, geom_data, q, not verbose)
    if verbose:
        print("Has collision {}".format(has_collision))
        i = 0
        for k in range(len(geom_model.collisionPairs)): 
          cr = geom_data.collisionResults[k]
          cp = geom_model.collisionPairs[k]
          if cr.isCollision():
              i = i+1
              print("collision pair:",cp.first,",",cp.second,"- collision:","Yes" if cr.isCollision() else "No")
        print(i)
    return has_collision

def jacobian(robot, q, tip = None, keep= None):
        
    if tip is None:
        tip = robot.model.frames[-1].name
    joint_id = model.getFrameId(tip)
    # joint_id = robot.model.getFrameId("r_wrist_ball_link")
    # joint_id =  model.getFrameId(robot.model.frames[-1].name)
    J = pin.computeFrameJacobian(robot.model,
                                       robot.data,
                                       q,
                                       joint_id,
                                       reference_frame=pin.LOCAL_WORLD_ALIGNED)[:3,:]
    
    if keep is None:
        u,s,v = np.linalg.svd(J)
    else:
        u,s,v = np.linalg.svd(J[:,keep])
    # print(J)
    return J, s

def dk(robot, q, tip = None):
    if tip is None:
        tip = robot.model.frames[-1].name
    # joint_id =  robot.model.getFrameId(robot.model.frames[-1].name)
    joint_id = robot.model.getFrameId(tip)
    pin.framesForwardKinematics(robot.model, robot.data, q)
    return robot.data.oMf[robot.model.getFrameId(tip)].translation.copy(), robot.data.oMf[robot.model.getFrameId(tip)].rotation

In [3]:
# use reachy's right arm with only 4 joints for now as they are the contribute 
# to its force capacity the most
# urdf_path = "reachy_v3_fix.urdf"
urdf_path = "reachy_v3_fix_shoulder.urdf"
# urdf_path = 'reachy_v3_simple.urdf'
# urdf_path = 'reachy.urdf'
robot = pin.RobotWrapper.BuildFromURDF(urdf_path)
model, data = robot.model, robot.data
# lock the other joints
tolock = [
   "l_shoulder_pitch",
   "l_shoulder_roll",
   "l_elbow_yaw",
   "l_elbow_pitch",
   "l_wrist_roll",
   "l_wrist_pitch",
   "l_wrist_yaw",
   "l_hand_finger",
   "l_hand_finger_mimic",
   "l_hand_finger_proximal",
   "l_hand_finger_distal",
   "l_hand_finger_proximal_mimic",
   "l_hand_finger_distal_mimic",
   "neck_roll",
   "neck_pitch",
   "neck_yaw",
   # "r_shoulder_dummy_1",
   # "r_shoulder_dummy_2",
   # "r_shoulder_dummy_out",
   # "r_shoulder_pitch",
   # "r_shoulder_roll",
   # "r_elbow_yaw",
   # "r_elbow_pitch",
   # "r_wrist_roll",
   # "r_wrist_pitch",
   # "r_wrist_yaw",
   "r_hand_finger",
   "r_hand_finger_mimic",
   "r_hand_finger_proximal",
   "r_hand_finger_distal",
   "r_hand_finger_proximal_mimic",
   "r_hand_finger_distal_mimic",
]

robot.model, robot.data, robot.collision_model = reduce_model(robot, tolock)
model, data = robot.model, robot.data

In [4]:
viewer = meshcat.Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
robot.data = robot.model.createData()

# get joint position ranges
q_max = robot.model.upperPositionLimit.T
q_min = robot.model.lowerPositionLimit.T
q_min[[2,3,5,6,7,8,9]] = np.array([-2*np.pi, -3.07, -2*np.pi, -2.22,-np.pi/4,-np.pi/4,-np.pi])
q_max[[2,3,5,6,7,8,9]] = np.array([2*np.pi, 0.37, 2*np.pi, 0.02,np.pi/4,np.pi/4,np.pi])
# q_min = np.array([-np.pi, -3.07, -np.pi, -2.22,-np.pi/4,-np.pi/4,-np.pi])[:robot.nq]
# q_max = np.array([np.pi, 0.37, np.pi, 0.02,np.pi/4,np.pi/4,np.pi])[:robot.nq]
robot.model.upperPositionLimit = q_max
robot.model.lowerPositionLimit = q_min
dq_max = 5*np.ones(robot.nq)
dq_min = -dq_max
# get max velocity
t_max = np.ones(robot.nq)*4 # amps
t_min = -t_max

# Use robot configuration.
q = robot.q0[:robot.nq]
# q = (q_min+q_max)/2

geom_model, geom_data = config_collision_model(robot)    

viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.collision_model)
# Start a new MeshCat server and client.
viz.initViewer(open=False, viewer = viewer)
# Load the robot in the viewer.
viz.loadViewerModel("reachy")
viz.display(q)
# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 
viz.display_collisions = True

In [6]:
from ipywidgets import interact, FloatSlider, Layout, Label

kwargs = {"\tq[{}]".format(i,name) : 
          FloatSlider(
              min = q_min[i], 
              max = q_max[i], 
              step = 0.01, 
              value = q[i],
          layout=Layout(width='300px'))
          for i,(q_1, name) in enumerate(zip(q, getJointNames(robot)))}
@interact(**kwargs)
def update(**kwargs):
    q = np.array([v  for v in kwargs.values()])
    viz.display(q)
    computeCollisions(robot, geom_model, q)

interactive(children=(FloatSlider(value=0.0, description='\tq[0]', layout=Layout(width='300px'), max=1.5707963…

# Experiment

In [ ]:
# from reachy2_symbolic_ik.symbolic_ik import SymbolicIK

# q_shoulder = [
#     [10, 0, 15, "beta"],
#     [0,0,0, "straight"],
#     [-10,0,0, "up10"],
#     [-20,0,0, "up20"],
#     [-20,0,10, "up20front10"],
#     [-20,0,15, "up20front15"]
# ]


# goal_grasps = [
#     [np.pi/2, -np.pi/2, 0, "side grasp"]
#     [0, 0, 0, "top grasp"],
#     [0, -np.pi/2, 0, "front grasp"],
#     [0, -np.pi, 0, "bottom grasp"]
# ]

# tolock = [
#    "r_shoulder_dummy_1",
#    "r_shoulder_dummy_2",
#    "r_shoulder_dummy_out"
#    ]

# reachable_all = []
# pos_all  = []
# q_list_all, q_lock_all = [], []
# for q_sh in q_shoulder:
    
#     symbolic_ik_r = SymbolicIK(
#         arm="r_arm",
#         backward_limit=-1, 
#         shoulder_orientation_offset = np.array(q_sh[:3]),
#         shoulder_position = np.array([0.0, -0.2, 0.0]),
#     )


#     q_lock = [np.deg2rad(q_sh[0]-10), np.deg2rad(q_sh[2]-15), 0.0]
#     print(q_lock)
#     model_r, dat_a_r, col_model_r = reduce_model(robot, tolock, q_lock)
#     q_lock_all.append(q_lock)
#     robot_r = pin.RobotWrapper(model_r, col_model_r)
    
#     geom_model_r, geom_data_r = config_collision_model(robot_r)
    
#     pos, q_list= [], []
#     reachable = []
#     N = 20
#     for x in np.linspace(-0.0,0.5,N):
#         for y in np.linspace(-0.5,0.4,N):
#             for z in np.linspace(0.45,1.35,N):
#                 goal_position = [x, y, z] - np.array([-0.0452,  0.    ,  1.    ])
#                 grasp_reachable, grasp_q= [], []
#                 for grasp in goal_grasps:
#                     grasp = grasp[:3] # don't use the grasp name
#                     # Check if the goal pose is reachable
#                     is_reachable, theta, getjoints = symbolic_ik_r.is_reachable(np.array([goal_position, grasp]))
#                     q_s = np.zeros(robot_r.nq)
#                     if is_reachable:
#                         is_good, q = check_rechable_with_collisions(robot_r, geom_model_r, theta, getjoints)
#                         is_reachable = is_good
#                         if is_good:
#                             q_s = q
#                     grasp_reachable.append(is_reachable)
#                     grasp_q.append(q_s)
#                 pos.append([x,y,z])
#                 reachable.append(grasp_reachable)
#                 q_list.append(grasp_q)
#     pos_all.append(pos)
#     reachable_all.append(reachable)
#     q_list_all.append(q_list)
    
# q_list_all = np.array(q_list_all)
# pos_all = np.array(pos_all)
# reachable_all = np.array(reachable_all, dtype=float)


[0.0, 0.0, 0.0]
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of range
wrist out of ra

# Load data

In [36]:
import pickle
# filename = "data/reachability_symik_7dof.pckl"
# with open(filename, 'wb') as f:
#     pickle.dump([q_list_all, pos_all, reachable_all, q_lock_all, q_shoulder, goal_grasps], f)

In [37]:
import pickle
filename = "data/reachability_symik_7dof.pckl"
with open(filename, 'rb') as f:
    q_list_all, pos_all, reachable_all, q_lock_all, q_shoulder, goal_grasps =  pickle.load(f)

In [38]:
q_shoulder

[[10, 0, 15, 'beta'],
 [0, 0, 0, 'straight'],
 [-10, 0, 0, 'up10'],
 [-20, 0, 0, 'up20'],
 [-20, 0, 10, 'up20front10']]

In [39]:
 goal_grasps

[[1.5707963267948966, -1.5707963267948966, 0, 'side grasp'],
 [0, 0, 0, 'top grasp'],
 [0, -1.5707963267948966, 0, 'front grasp'],
 [0, -3.141592653589793, 0, 'bottom grasp']]

# Visualise

In [47]:
## visualise the polytope and the ellipsoid
import meshcat.geometry as g 

model_i = getModelIds(q_shoulder,'beta')[0]
grasp_i = getGraspIds(goal_grasps, "top grasp")[0]


grasp_i1 = getGraspIds(goal_grasps, "side grasp")[0]

reachable = reachable_all[model_i]
pos = pos_all[model_i]

reachability = np.sum(reachable[:,[grasp_i,grasp_i1]],axis=1) # grasp union
# reachability = np.prod(reachable[:,[grasp_i,grasp_i1]],axis=1) # intersection
# reachability = reachable[:,grasp_i] # one grasp

pos_reachable_only = pos[reachability>0] 
reachable_only = reachability[reachability>0]
# viz.viewer["point_cloud"].set_object(g.PointCloud(position=pos.T, color=np.array([1-reachability/reachable_only.max(), reachability/reachability.max(), np.zeros_like(reachability)]),size=0.02))
viz.viewer["point_cloud"].set_object(g.PointCloud(position=pos_reachable_only.T, color=np.array([1-reachable_only/reachable_only.max(), reachable_only/reachable_only.max(), np.zeros_like(reachable_only)]),size=0.02))
from reachable_space_utils  import *
verts, faces = alpha_shape_with_cgal(pos_reachable_only, alpha=0.1)
vert = faces.reshape(-1,3)
faces = np.arange(len(vert)).reshape(-1,3)
poly = g.TriangularMeshGeometry(vertices=vert, faces=faces)
viz.viewer['poly'].set_object(poly, g.MeshBasicMaterial(color=0x000000, wireframe=False, linewidth=2, opacity=0.2))